In [ ]:
import pandas as pd
import glob
import json
import matplotlib.pyplot as plt
import numpy as np

# Preprocessing light curves from the ellipticals Galaxy Catalogs

In [16]:
file_list = glob.glob('/media3/CRP8/TDE/data/DR_photometry/ellipticals/*.json')
file_list.sort()
print('Number of unique Objects (ZTF_ID):',len(file_list), file_list[0])

Number of unique Objects (ZTF_ID): 1094 /media3/CRP8/TDE/data/DR_photometry/ellipticals/ZTF17aaaprdf.json


#### Exploring data

In [17]:
with open(file_list[0], "r") as f:
    data = json.load(f)
    
data['753108100007817']['meta']

{'nobs': 699,
 'ngoodobs': 614,
 'duration': 2407.229250000004,
 'filter': 'zg',
 'fieldid': 753,
 'rcid': 28,
 'coord': {'ra': 163.65847778320312, 'dec': 47.47028732299805},
 'h3': {'10': 621893209549144063}}

In [42]:
data_alert = pd.read_parquet('/media3/CRP8/TDE/data/xmatch/ellipticals/')
data_alert = data_alert.groupby('objectId', sort=False, as_index=False).agg(list)
data_alert['top_class'] = [np.unique(data_alert['finkclass'].values[i]) for i in range(data_alert.shape[0])]

print(data_alert.columns)
data_alert.head()

Index(['objectId', 'finkclass', 'candid', 'dec', 'fid', 'jd', 'magpsf', 'ra',
       'sigmapsf', 'OBJID', 'top_class'],
      dtype='object')


,objectId,finkclass,candid,dec,fid,jd,magpsf,ra,sigmapsf,OBJID,top_class
0,ZTF18aaiiwwc,"[Unknown, Unknown, Unknown, Unknown, Unknown]","[1149498284915010005, 1479494280715010004, 153...","[29.6218091, 29.621942, 29.6217332, 29.6215087...","[2, 2, 2, 2, 2]","[2458903.998287, 2459233.9942824, 2459292.8902...","[19.547908782958984, 19.347326278686523, 19.70...","[221.1094436, 221.1093877, 221.1094004, 221.10...","[0.18840081989765167, 0.1332128494977951, 0.17...","[587739407332540598, 587739407332540598, 58773...",[Unknown]
1,ZTF20acynbsx,[BLLac],[1448145345315015001],[15.4401491],[2],[2459202.6453472],[20.08265495300293],[15.8583704],[0.20976054668426514],[587724233713909815],[BLLac]
2,ZTF18aaiwsqa,"[Seyfert_1, Seyfert_1, Seyfert_1, Seyfert_1, S...","[1595323621415010024, 1561321505715010000, 120...","[29.813235, 29.8132183, 29.8131265, 29.8132112...","[2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, ...","[2459349.8236227, 2459315.8215046, 2458956.838...","[18.85457420349121, 18.91497230529785, 18.5430...","[239.1783763, 239.1784741, 239.1786393, 239.17...","[0.13848237693309784, 0.13321274518966675, 0.1...","[587736919431774294, 587736919431774294, 58773...",[Seyfert_1]
3,ZTF19aailuwx,"[Seyfert_1, Seyfert_1, Seyfert_1, Seyfert_1, S...","[1700183111715015000, 1982347281715015002, 123...","[25.9871184, 25.9871419, 25.9871296, 25.987190...","[1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, ...","[2459454.6831134, 2459736.8472801, 2458985.810...","[19.748014450073242, 20.035320281982422, 20.25...","[235.9807616, 235.9807525, 235.980711, 235.980...","[0.15512745082378387, 0.18819434940814972, 0.2...","[587736976348086286, 587736976348086286, 58773...",[Seyfert_1]
4,ZTF18aaowkhh,"[Unknown, Unknown]","[1986342464815010003, 1973372174815010011]","[29.8135792, 29.8134409]","[2, 1]","[2459740.8424653, 2459727.8721759]","[19.05344009399414, 19.581531524658203]","[244.6567474, 244.6569692]","[0.13012142479419708, 0.19437047839164734]","[587735744228556860, 587735744228556860]",[Unknown]


### Preprocessing Light curves

In [25]:
all_lcs = []

# Filter mapping
filter_map = {"zg": ("g", 1), "zr": ("r", 2), "zi": ("i", 3)}
count = 0
for file in file_list:
    with open(file, "r") as f:
        data = json.load(f)

    count = count+1
    
    ztf_id = file.split("/")[-1].split(".")[0]
    
    # Loop over different matches (keys)
    for obj_id in data.keys():
        lc = data[obj_id]["lc"]
        filt = data[obj_id]["meta"]["filter"]  # e.g., "zg"
        
        if filt not in filter_map:
            continue 
        
        band, fid = filter_map[filt]
        
        # Extract points
        for point in lc:
            all_lcs.append({
                "ztf_id": ztf_id,
                "obj_id": obj_id,
                "mjd": point["mjd"],
                "mag": point["mag"],
                "magerr": point["magerr"],
                "band": band,
                "fid": fid
            })

# Convert to dataframe
df = pd.DataFrame(all_lcs)
print(df.head())
print(df["band"].value_counts())

         ztf_id           obj_id          mjd        mag    magerr band  fid
0  ZTF17aaaprdf  753108100007817  58202.28430  16.014210  0.013256    g    1
1  ZTF17aaaprdf  753108100007817  58205.26971  16.082413  0.013400    g    1
2  ZTF17aaaprdf  753108100007817  58205.31793  16.044706  0.013319    g    1
3  ZTF17aaaprdf  753108100007817  58206.26646  16.056454  0.013344    g    1
4  ZTF17aaaprdf  753108100007817  58206.28801  16.115350  0.013474    g    1
band
r    1006318
g     779538
i     194341
Name: count, dtype: int64


In [26]:
count

1094

In [24]:
data_lc = df.groupby('ztf_id', sort=False, as_index=False).agg(list)
data_lc

,ztf_id,obj_id,mjd,mag,magerr,band,fid
0,ZTF17aaaprdf,"[753108100007817, 753108100007817, 75310810000...","[58202.2843, 58205.26971, 58205.31793, 58206.2...","[16.01421, 16.082413, 16.044706, 16.056454, 16...","[0.013256412, 0.013399877, 0.013318792, 0.0133...","[g, g, g, g, g, g, g, g, g, g, g, g, g, g, g, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,ZTF17aaaprse,"[671103300014360, 671103300014360, 67110330001...","[58202.33691, 58205.28701, 58205.30855, 58227....","[15.902462, 15.945415, 15.957567, 15.818044, 1...","[0.016709503, 0.016774489, 0.016793802, 0.0165...","[g, g, g, g, g, g, g, g, g, g, g, g, g, g, g, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,ZTF17aaapuge,"[626111300000845, 626111300000845, 62611130000...","[58202.31486, 58202.33978, 58205.30975, 58205....","[16.194317, 16.166513, 16.259598, 16.234364, 1...","[0.01437236, 0.014289263, 0.014580228, 0.01449...","[g, g, g, g, g, g, g, g, g, g, g, g, g, g, g, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,ZTF17aaaywow,"[567114400012027, 567114400012027, 56711440001...","[58205.15615, 58227.17411, 58233.22722, 58236....","[15.434539, 15.501621, 15.6012745, 15.517904, ...","[0.011071682, 0.011127067, 0.011224774, 0.0111...","[g, g, g, g, g, g, g, g, g, g, g, g, g, g, g, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,ZTF17aaazdba,"[616301300021581, 616301300021581, 61630130002...","[58936.25228, 59180.41628, 59190.39434, 59199....","[15.447303, 15.291398, 15.498183, 15.457579, 1...","[0.011534797, 0.011246867, 0.01164372, 0.01155...","[i, i, i, i, i, i, i, i, i, i, i, i, i, i, i, ...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
...,...,...,...,...,...,...,...
1085,ZTF24abcqbxo,"[1660110400008114, 1660110400008114, 166011040...","[58427.38352, 58430.3738, 58438.42597, 58439.4...","[15.783054, 15.966704, 15.9584255, 15.883247, ...","[0.010763775, 0.011099269, 0.011082455, 0.0109...","[g, g, g, g, g, g, g, g, g, g, g, g, g, g, g, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1086,ZTF24abgisqm,"[464115200003718, 464115200003718, 46411520000...","[58206.13588, 58206.17056, 58227.17224, 58235....","[17.759686, 17.744753, 17.689104, 17.668892, 1...","[0.025289426, 0.025077522, 0.024312016, 0.0240...","[g, g, g, g, g, g, g, g, g, g, g, g, g, g, g, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1087,ZTF24abgitfe,"[665107100004382, 665107100004382, 66510710000...","[58202.26618, 58205.25936, 58235.1983, 58238.1...","[19.739323, 19.626102, 19.891449, 19.54081, 19...","[0.0983955, 0.09061788, 0.10963237, 0.08509850...","[g, g, g, g, g, g, g, g, g, g, g, g, g, g, g, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1088,ZTF24abpnljh,"[665108300003137, 665108300003137, 66510830000...","[58202.26603, 58205.2592, 58235.19809, 58238.1...","[17.658472, 17.68474, 17.630451, 17.71821, 17....","[0.022414915, 0.02272704, 0.022090334, 0.02313...","[g, g, g, g, g, g, g, g, g, g, g, g, g, g, g, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


### Catalog matching with alerts

In [32]:
data_alert['ztf_id'] = data_alert['objectId']

In [33]:
data_all = pd.merge(data_lc, data_alert, on="ztf_id", how="inner")

In [44]:
data_all.to_csv('ellipticals_DR_Alert.csv')

### Checking if a ZTF Id is uniquely associated to one object in the catalog

In [46]:
data_all['flag_multiple_source'] = 0

for i in range(len(data_all)):
    ztf_id = data_all.loc[i, 'ztf_id']
    cats = np.unique(data_all.loc[data_all['ztf_id'] == ztf_id, 'OBJID'])

    if len(cats) > 2:
        data_all.loc[i, 'flag_multiple_source'] = 1

In [47]:
data_all['flag_multiple_source'].value_counts()

flag_multiple_source
0    1090
Name: count, dtype: int64

In [ ]:
data_alert['top_class'].value_counts()

In [60]:
data_alert.loc[0, 'OBJID']

['587739407332540598',
 '587739407332540598',
 '587739407332540598',
 '587739407332540598',
 '587739407332540598']

In [61]:
for i in range(len(data_alert)):
    print(np.unique(data_alert.loc[i, 'OBJID']))
          

['587739407332540598']
['587724233713909815']
['587736919431774294']
['587736976348086286']
['587735744228556860']
['587732580982521871']
['587731892189855768']
['587741816244797628']
['587742773483274278']
['588017570854207514']
['587729748448314652']
['587731892188610595']
['587732134306578454']
['587742013827383378']
['587741828579983395']
['587741828046323753']
['587725819100397757']
['587724240159441069']
['588017720103403534']
['588848898830237807']
['588017603610017803']
['587736543096602645']
['588018056187543590']
['587745968928129047']
['587726032787996684']
['587741830194724900']
['588017721177276566']
['588010358546104373']
['587745402007060524']
['588016893323903061']
['587727220862943574']
['587738618103922723']
['587741490911576146']
['587737808493936696']
['587730775498489910']
['587745243615133959']
['587728879269642276']
['588018089466986540']
['587738067265323344']
['587742566779912214']
['587739811032989796']
['587736942525481001']
['587741532244803602']
['587732156

In [ ]:
# data_alert['OBJID_unique'] = np.unique(data_alert['OBJID_unique'])
# data_alert_obj = data_alert.groupby('OBJID', sort=False, as_index=False).agg(list)
# data_alert_obj